## Setup

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_gan as tfgan
import numpy as np
import os, sys
from tqdm.notebook import tqdm
from pathlib import Path

sys.path.append( os.path.abspath('..') )
import utils

In [ ]:
Path('Fashion').mkdir(exist_ok=True)
os.chdir('Fashion')

In [ ]:
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()
data = np.concatenate((x_train, x_test))
data = (data.astype('float32') - 127.5) / 127.5  # normalize to [-1, 1]
data = np.expand_dims(data, axis=-1)  # add channels dimension

assert data.shape == (70000, 28, 28, 1)  # (batch, height, width, channel)

## 1 Models

### 1.1 Architecture

In [ ]:
def generator_model(latent_dims):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(7*7*256, input_shape=(latent_dims,)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Reshape((7, 7, 256)),
    
        tf.keras.layers.Conv2D(128, kernel_size=5, strides=1, padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.UpSampling2D(size=2, interpolation='bilinear'),
        tf.keras.layers.Conv2D(64, kernel_size=5, strides=1, padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.UpSampling2D(size=2, interpolation='bilinear'),
        tf.keras.layers.Conv2D(1, kernel_size=5, strides=1, padding='same', activation='tanh')
    ])
    return model

In [ ]:
def critic_model(constraint):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            64, kernel_size=5, strides=2, padding='same', input_shape=(28,28,1),
            kernel_constraint=constraint, bias_constraint=constraint
        ),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(
            128, kernel_size=5, strides=2, padding='same',
            kernel_constraint=constraint, bias_constraint=constraint
        ),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, kernel_constraint=constraint, bias_constraint=constraint)
    ])
    return model

### 1.2 Losses

For the Wasserstein metric, the loss for the generator (G) given the critic (f) is given by:

$$
    -\mathbb{E}_{z \sim p(z)}\bigl\lbrack f\bigl(G(z)\bigr) \bigr\rbrack
$$

In [ ]:
def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

The critic loss tries to maximize:
$$
    \max_{\|f \|_{L} \leq 1} \mathbb{E}_{x \sim p_{data}} \bigl\lbrack f(x) \bigr\rbrack - 
    \mathbb{E}_{z \sim p_{z}} \bigl\lbrack f\bigl(G(z)\bigr) \bigr\rbrack
$$

Which is equivalent to minimizing the negative of this value, as shown in the function bellow

In [ ]:
def critic_loss(real_output, fake_output):
    mu_real = tf.reduce_mean(real_output)
    mu_fake = tf.reduce_mean(fake_output)
    return mu_fake - mu_real

## 2 Training

### 2.1 Main functions

In [ ]:
def critic_train_step(generator, critic, images, latent_dims):
    noise = tf.random.normal([images.shape[0], latent_dims])
    with tf.GradientTape() as crit_tape:
        generated_imgs = generator(noise, training=True)
        real_output = critic(images, training=True)
        fake_output = critic(generated_imgs, training=True)
        loss_C = critic_loss(real_output, fake_output)
    
    grads_C = crit_tape.gradient(loss_C, critic.trainable_variables)
    critic.optimizer.apply_gradients(zip(grads_C, critic.trainable_variables))

In [ ]:
def generator_train_step(generator, critic, batch_size, latent_dims):
    noise = tf.random.normal([batch_size, latent_dims])
    with tf.GradientTape() as gen_tape:
        generated_imgs = generator(noise, training=True)
        fake_output = critic(generated_imgs, training=True)
        loss_G = generator_loss(fake_output)
    
    grads_G = gen_tape.gradient(loss_G, generator.trainable_variables)
    generator.optimizer.apply_gradients(zip(grads_G, generator.trainable_variables))

In [ ]:
def train(generator, critic, data, epochs, n_critic=1, batch_size=32, callbacks=None):
    latent_dims = generator.input_shape[1]
    dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(data.shape[0]).batch(batch_size)
    iterator     = iter(dataset)
    num_batches  = 1 + (data.shape[0] - 1) // batch_size
    batches_left = True
    batch_count  = 0
    
    generator_step = tf.function(generator_train_step)
    critic_step = tf.function(critic_train_step)
    for epoch in tqdm(range(epochs)):
        for c in callbacks:
            c.on_epoch_begin(epoch=epoch + 1, generator=generator, discriminator=critic)
        
        batch_pbar = tqdm(total=num_batches, leave=False)
        while batches_left:
            for i in range(n_critic):
                if batch_count == num_batches:
                    batch_count = 0
                    batches_left = False
                    iterator = iter(dataset)
                batch_count += 1
                batch_pbar.update()
                batch = iterator.get_next() 
                critic_step(generator, critic, batch, latent_dims)
            generator_step(generator, critic, batch_size, latent_dims)
        batches_left = True
        batch_pbar.update(num_batches)
        batch_pbar.close()
        
        for c in callbacks:
            c.on_epoch_end(epoch=epoch + 1, generator=generator, discriminator=critic)

### 2.2 Metrics classifier

Loading the classifier that will be used to calculate the *Classifier Score* (CS) and *Fréchet Classifier Distance* (FCD). \
The features of the real data are also precalculated to avoid doing that for each epoch.

In [ ]:
classifier = tf.keras.models.load_model('../../Classifiers/fashion.h5')
feature_layer = classifier.get_layer('features')
logits_layer = classifier.get_layer('logits')
precalculated_features = utils.fn.calculate_features(classifier, feature_layer, data)

### 2.3 Hyperparameter Testing

This class implements the *weight clipping* constraint used to guarantee the critic is a K-Lipschitz continuous function.

In [ ]:
class ClipConstraint(tf.keras.constraints.Constraint):
    def __init__(self, clip_value):
        self.__clip_value = clip_value

    def __call__(self, w):
        return tf.clip_by_value(w, -self.__clip_value, self.__clip_value)

These were the hyperparameters tested for the final document. Training all of them simultaneously may take a long time, consider commenting out some options to run the tests individually.

In [ ]:
LATENT_DIMS = 24
BATCH_SIZE = 32

In [ ]:
hparams_list = [
    {'n_critic':  5, 'clip_value': 0.01, 'learning_rate': 5e-5},
    {'n_critic':  5, 'clip_value': 0.50, 'learning_rate': 5e-5},
    {'n_critic':  5, 'clip_value': 0.99, 'learning_rate': 5e-5},
    {'n_critic':  5, 'clip_value': 0.99, 'learning_rate': 1e-3},
    {'n_critic':  5, 'clip_value': 0.99, 'learning_rate': 5e-3},
    {'n_critic':  5, 'clip_value': 0.99, 'learning_rate': 2e-4},
    {'n_critic':  5, 'clip_value': 2.00, 'learning_rate': 5e-5},
    {'n_critic': 10, 'clip_value': 0.99, 'learning_rate': 1e-3}
]

In [ ]:
for hparams in hparams_list:
    dirname = 'NCRIT{}_CLIP{}_LR{:.0e}'.format(
        hparams['n_critic'],
        hparams['clip_value'],
        hparams['learning_rate']
    )
    Path(dirname).mkdir(exist_ok=True)
    
    generator = generator_model(LATENT_DIMS)
    # generator.optimizer = tf.keras.optimizers.Adam(hparams['learning_rate'], beta_1=0)
    generator.optimizer = tf.keras.optimizers.RMSprop(hparams['learning_rate'])
    
    constraint = ClipConstraint(hparams['clip_value'])
    critic = critic_model(constraint)
    # critic.optimizer = tf.keras.optimizers.Adam(hparams['learning_rate'], beta_1=0)
    critic.optimizer = tf.keras.optimizers.RMSprop(hparams['learning_rate'])
    
    ## Callbacks
    timer = utils.callback.TimerCallback()
    save_samples = utils.callback.SaveSamplesCallback(
        path_format=os.path.join(dirname, 'epoch-{}'),
        inputs=tf.random.normal((10*10, LATENT_DIMS)),
        n_cols=10,
        savefig_kwargs={'bbox_inches': 'tight', 'pad_inches': 0, 'dpi': 192},
        imshow_kwargs={'cmap': 'gray_r', 'vmin': -1, 'vmax': 1}
    )
    metrics = utils.callback.MetricsCallback(
        generator=generator,
        classifier=classifier,
        latent_dims=LATENT_DIMS,
        feature_layer=feature_layer,
        logits_layer=logits_layer,
        precalculated_features=precalculated_features,
        save_after=5, save_to=os.path.join(dirname, 'best.h5')
    )
    
    ## Train and save results
    train(
        generator, critic, data, epochs=20,
        n_critic=hparams['n_critic'],
        batch_size=BATCH_SIZE,
        callbacks=[timer, save_samples, metrics]
    )
    
    metrics_obj = metrics.get_metrics()
    metrics_obj['time'] = timer.get_time()
    utils.fn.update_json_log(os.path.join(dirname, 'log.json'), metrics_obj)
    
    generator.save(os.path.join(dirname, 'generator.h5'), overwrite=True, save_format='h5')
    critic.save   (os.path.join(dirname, 'critic.h5'   ), overwrite=True, save_format='h5')

\
In windows the command bellow is used to turn down the machine after the training finishes, very useful if you wanna let the computer running while you go to sleep :)

In [ ]:
# !shutdown /s /t 60